In [43]:
import psycopg2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime as dt

import seaborn as sns
import datetime

##DataFrames To be used for analysis of infielders
#Reguler_Inflder_Stats_30.csv   *played atleast 30 games
#Reguler_Inflder_Stats_100.csv  *played atleast 100 games
#All_Star_Inflder_Stats_100.csv  *alstars who played atleast 100 gms

In [44]:
import psycopg2 as p2
conn = p2.connect(database = 'player_stats', user ='baseball_master', password = 'georgetowndata', 
                  host = 'georgetown-baseball.cnfqonxqdbry.us-east-1.rds.amazonaws.com', 
                  port = '5432')

### Non- All Star Infielders who have played at least 30 games at their position


In [45]:
query="""
select "playerID", "POS", "yearID", "GS", "G"
from fielding 
where "yearID" >= '1947'
and "POS" in ('2B','3B','1B', 'SS')
and "G" > 30 and "G" < 100
and "playerID" not in 
    (select "playerID"
    from fielding 
    where "GS" > 100
    and "POS" in ('2B','3B','1B','SS')
    and "playerID" not in 
        (Select "playerID"  ---count("playerID")
        from allstar_full
        group by "playerID"
        having count("playerID") > 5))
"""

Reguler_Inflder_30 = pd.read_sql_query(query, conn)

In [46]:
Reguler_Inflder_30.head(3)

,playerID,POS,yearID,GS,G
0,adamsbo03,2B,1947.0,NaN,69.0
1,albrija01,SS,1947.0,NaN,33.0
2,basined01,2B,1947.0,NaN,56.0


### Batting Stats for Non ALstars Infielders who played atleast 30 games

In [47]:
query="""
select a."playerID", m."nameFirst", m."nameLast", m."birthYear", m."deathYear", a."yearID", a."G", a."teamID", a."AB",a."H",a."2B", a."3B",
a."HR", a."SO", a."HBP", a."BB", a."SF"
from batting a
join master m
on a."playerID" = m."playerID"
where a."yearID" >= '1947'
"""

Batting_Stats = pd.read_sql_query(query,conn)

In [48]:
Batting_Stats['AVG'] = Batting_Stats['H'] / Batting_Stats['AB']

In [49]:
Batting_Stats["1B"] = Batting_Stats["H"] - Batting_Stats["2B"]- Batting_Stats["3B"]
- Batting_Stats["HR"]

0        -0.0
1        -4.0
2        -4.0
3        -2.0
4        -2.0
5        -2.0
6        -0.0
7        -8.0
8        -0.0
9        -0.0
10       -0.0
11       -0.0
12       -0.0
13       -0.0
14       -1.0
15       -0.0
16       -0.0
17       -0.0
18       -4.0
19       -1.0
20       -5.0
21       -0.0
22       -0.0
23       -0.0
24       -0.0
25       -0.0
26       -0.0
27       -0.0
28       -0.0
29       -0.0
         ... 
70052    -0.0
70053    -3.0
70054    -5.0
70055    -0.0
70056    -0.0
70057    -0.0
70058    -0.0
70059    -0.0
70060    -7.0
70061    -0.0
70062    -0.0
70063    -0.0
70064    -0.0
70065    -0.0
70066   -21.0
70067    -0.0
70068    -0.0
70069    -0.0
70070    -0.0
70071    -9.0
70072    -0.0
70073    -0.0
70074    -0.0
70075    -0.0
70076    -0.0
70077    -0.0
70078   -15.0
70079   -18.0
70080   -12.0
70081    -0.0
Name: HR, Length: 70082, dtype: float64

In [50]:
Batting_Stats.HBP.fillna(0, inplace = True)  #replace NaN with zero

In [51]:
Batting_Stats.SF.fillna(0, inplace = True)  #replace NaN with zero

In [52]:
Batting_Stats["TB"] = Batting_Stats["1B"] + Batting_Stats["2B"]*2 
+ Batting_Stats["3B"]*3 + Batting_Stats["HR"] * 4

0         0.0
1        25.0
2        22.0
3        11.0
4        17.0
5         8.0
6         0.0
7        32.0
8         0.0
9         0.0
10        0.0
11        0.0
12        0.0
13        9.0
14        4.0
15        0.0
16        0.0
17        0.0
18       22.0
19        4.0
20       47.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        0.0
27        0.0
28        0.0
29        0.0
         ... 
70052     0.0
70053    18.0
70054    41.0
70055     0.0
70056     0.0
70057     0.0
70058     0.0
70059     0.0
70060    28.0
70061     0.0
70062     0.0
70063     0.0
70064     0.0
70065     0.0
70066    93.0
70067     0.0
70068     0.0
70069     0.0
70070     0.0
70071    36.0
70072     0.0
70073     0.0
70074     0.0
70075     0.0
70076     0.0
70077     0.0
70078    63.0
70079    81.0
70080    48.0
70081     0.0
Length: 70082, dtype: float64

In [53]:
Batting_Stats["SLG"] = Batting_Stats["TB"] / Batting_Stats["AB"]

In [54]:
Batting_Stats["OBP"] = (Batting_Stats["H"] + Batting_Stats["BB"] 
+ Batting_Stats["HBP"]) / (Batting_Stats ["AB"] + Batting_Stats ["BB"]
+ Batting_Stats["HBP"] + Batting_Stats["SF"])


In [55]:
Batting_Stats.head(5)

,playerID,nameFirst,nameLast,birthYear,deathYear,yearID,G,teamID,AB,H,...,HR,SO,HBP,BB,SF,AVG,1B,TB,SLG,OBP
0,abernwo01,Woody,Abernathy,1915.0,1994.0,1947.0,1.0,NY1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN
1,aberscl01,Cliff,Aberson,1921.0,1973.0,1947.0,47.0,CHN,140.0,39.0,...,4.0,32.0,0.0,20.0,0.0,0.278571,30.0,42.0,0.300000,0.368750
2,adamsbo03,Bobby,Adams,1921.0,1997.0,1947.0,81.0,CIN,217.0,59.0,...,4.0,23.0,4.0,25.0,0.0,0.271889,46.0,68.0,0.313364,0.357724
3,adamsbu01,Buster,Adams,1915.0,1990.0,1947.0,69.0,PHI,182.0,45.0,...,2.0,29.0,0.0,26.0,0.0,0.247253,33.0,55.0,0.302198,0.341346
4,adamsdi01,Dick,Adams,1920.0,2016.0,1947.0,37.0,PHA,89.0,18.0,...,2.0,18.0,0.0,2.0,0.0,0.202247,13.0,17.0,0.191011,0.219780


In [56]:
Batting_Stats["OPS"] = Batting_Stats["OBP"] + Batting_Stats["SLG"]

### Merge Infielder Dataframe with Batting Avg Dataframe

In [57]:
Reguler_Inflder_Stats_30= pd.merge(Reguler_Inflder_30, Batting_Stats, on = ["playerID", "yearID"])

In [58]:
Reguler_Inflder_Stats_30.head(3)

,playerID,POS,yearID,GS,G_x,nameFirst,nameLast,birthYear,deathYear,G_y,...,SO,HBP,BB,SF,AVG,1B,TB,SLG,OBP,OPS
0,adamsbo03,2B,1947.0,NaN,69.0,Bobby,Adams,1921.0,1997.0,81.0,...,23.0,4.0,25.0,0.0,0.271889,46.0,68.0,0.313364,0.357724,0.671088
1,albrija01,SS,1947.0,NaN,33.0,Jack,Albright,1921.0,1991.0,41.0,...,11.0,0.0,10.0,0.0,0.232323,19.0,27.0,0.272727,0.302752,0.575480
2,basined01,2B,1947.0,NaN,56.0,Eddie,Basinski,1922.0,NaN,56.0,...,27.0,0.0,18.0,0.0,0.198758,24.0,36.0,0.223602,0.279330,0.502932


In [59]:
Reguler_Inflder_Stats_30.to_csv('Reguler_Inflder_Stats_30.csv') #csv file

### Non- All Star Infielders who have played at least 100 games at their position and Batting Stats

In [60]:
query="""

select "playerID", "POS", "yearID", "GS","G"
from fielding 
where "G" > 100
and "yearID" >= '1947'
and "POS" in ('2B','3B','1B','SS')
and "playerID" not in 
	(Select "playerID"  ---count("playerID")
from allstar_full
group by "playerID"
having count("playerID") > 5)
"""

Reguler_Inflder_100 = pd.read_sql_query(query, conn)

In [61]:
Reguler_Inflder_Stats_100 = pd.merge(Reguler_Inflder_100, Batting_Stats, on = ["playerID", "yearID"])

In [62]:
Reguler_Inflder_Stats_100.head(3)

,playerID,POS,yearID,GS,G_x,nameFirst,nameLast,birthYear,deathYear,G_y,...,SO,HBP,BB,SF,AVG,1B,TB,SLG,OBP,OPS
0,bakerfl01,3B,1947.0,NaN,101.0,Floyd,Baker,1916.0,2004.0,105.0,...,28.0,0.0,66.0,0.0,0.264151,83.0,107.0,0.288410,0.375286,0.663696
1,chrisma01,SS,1947.0,NaN,106.0,Mark,Christman,1913.0,1976.0,110.0,...,16.0,2.0,32.0,0.0,0.221925,66.0,96.0,0.256684,0.286765,0.543449
2,coxbi02,SS,1947.0,NaN,129.0,Billy,Cox,1919.0,1978.0,132.0,...,28.0,1.0,29.0,0.0,0.274102,108.0,168.0,0.317580,0.313059,0.630639


In [63]:
Reguler_Inflder_Stats_100.to_csv("Reguler_Inflder_Stats_100.csv")

### All Star Infielders who have plaved in over 5 All Star games and over 100 games

In [64]:
query="""
select "playerID", "POS", "yearID", "GS", "G"
from fielding 
where "yearID" >= '1947'
and "POS" in ('2B','3B','1B', 'SS')
and "G" > 100
and "playerID" in  
        (Select "playerID"  ---count("playerID")
        from allstar_full
        group by "playerID"
        having count("playerID") > 5)
"""

All_Star_Inflder_100 = pd.read_sql_query(query, conn)

In [65]:
All_Star_Inflder_Stats_100 = pd.merge(All_Star_Inflder_100, Batting_Stats, on = ["playerID", "yearID"])

In [66]:
All_Star_Inflder_Stats_100.head(2)

,playerID,POS,yearID,GS,G_x,nameFirst,nameLast,birthYear,deathYear,G_y,...,SO,HBP,BB,SF,AVG,1B,TB,SLG,OBP,OPS
0,applilu01,SS,1947.0,NaN,129.0,Luke,Appling,1907.0,1991.0,139.0,...,28.0,1.0,64.0,0.0,0.306163,125.0,183.0,0.363817,0.385563,0.749380
1,doerrbo01,2B,1947.0,NaN,146.0,Bobby,Doerr,1918.0,NaN,146.0,...,47.0,0.0,59.0,0.0,0.258467,112.0,158.0,0.281640,0.329032,0.610672


In [67]:
All_Star_Inflder_Stats_100.to_csv("All_Star_Inflder_Stats_100.csv")